#### Resources: [Parent Document Retriever.](https://python.langchain.com/docs/how_to/parent_document_retriever/)




In [1]:
%%capture
!pip install  -U langchain torch langchain_community transformers sentence-transformers PyPDF2  chromadb faiss-cpu

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `"meta-llama/Meta-Llama-3.1-8B"` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to

In [3]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [4]:
import torch
from PyPDF2 import PdfReader
from transformers import AutoModelForCausalLM,AutoTokenizer,pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


In [5]:
file_path = '/content/VIGGO.pdf'
reader = PdfReader(file_path)
documents = []
for page in reader.pages:
    text = page.extract_text()
    documents.append(Document(page_content=text, metadata={"source": file_path}))

### Retrieving larger chunks

In [6]:
# create the child documents:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

### Embeddings

In [7]:

model_name = "sentence-transformers/all-MiniLM-l6-v2"
device = "cpu"
normalize = False

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": normalize}
)


<ipython-input-7-6a9a5eaa76f9>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Indexing

In [8]:
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)

<ipython-input-8-13e2ad6363d3>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)


## Retriever

In [9]:
store = InMemoryStore()

In [10]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [11]:
retriever.add_documents(documents)

In [12]:
list(store.yield_keys())

['5994fad3-b3a0-411d-97fa-0798a4a868bc',
 '3a246307-1dc5-4b2a-98ec-9d4bb851cd65',
 'b307b55f-233e-40a1-9fd6-fa39b2448ba0',
 'd05a9e6e-50a3-4ae6-9516-a891df4d7e71',
 'f4de6852-1e54-403b-8045-6316cb6f1579',
 'de5f3762-2b25-4d85-be1b-aeccf4bac29a',
 '64562e1b-bd18-4c5d-b267-34e55ff4aa79',
 '5a21d4a6-912e-40e2-9245-f44935f0d5f1',
 'b19d1e07-1ebc-41f4-ae01-106bc309d753',
 '0af5062f-879f-4dc1-9650-2533206487dc',
 '9b3b6f8a-9b3b-4e15-8bea-773c202ffe3f',
 '4d533017-c68e-440f-b6da-870a28f85e0c',
 '239c34ae-5249-43dd-9000-9bf8928167fc',
 '9bbf3334-c3fe-4f97-ba44-1f3d2c8c517c',
 'e1b82442-f9fd-4158-9eef-301f5dde167f',
 '02a983d1-0708-47ad-b53f-15765e48fa27',
 'b3b657d5-57c7-44d3-a0e6-4070091200f2',
 'cf24deb3-e351-4142-8bc5-62d95bf0d8fb',
 '9695e030-66bd-4c9b-92ba-dbd903687fcf',
 '88f9df6d-09bd-467f-97db-fe6d50347301',
 '5a990b16-6880-414d-8063-b8d038a7de49',
 '7a43ce0c-53e7-4e5f-81c9-436775ea2160',
 '6108f9ae-a0ff-496e-ac19-c3d8ce8efd0c',
 '3e4ab251-c057-4f02-8bd5-82f018472e0c',
 '1f0662ea-f77f-

## RAG Chain

### LLM

In [13]:
model_name = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./cache")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="./cache",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,     # Enable sampling
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-13-6f213a13110d>:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [14]:
template = """"
You are a helpful assistant that answers questions based on the following context
Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# Querying
response = rag_chain.invoke("How does the Transformer-based sequence-to-sequence model perform on the ViGGO dataset compared to other datasets like E2E and Hotel?")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Human: "
You are a helpful assistant that answers questions based on the following context
Context: [Document(metadata={'source': '/content/VIGGO.pdf'}, page_content='InstancesUnique\nMRsUnique\ndelex.\nMRsVocabDelex.\nvocabAvg. 3-\ngram\nfreq.Refs/\nMRSlots/\nMRW/\nRefW/\nSentSents/\nRef\nE2E 51,426 6,039 5,963 2,878 2,818 18.70 8.1 5.43 22.41 14.36 1.56\nViGGO inf 3,000 1,000 997 1,378 1,102 8.33 3 5.81 30.62 15.01 2.04\nViGGO 6,900 2,253 2,066 2,427 2,178 6.91 3 4.18 25.01 15.04 1.66\nTable 3: Dataset statistics comparing the ViGGO dataset, as well as its subset of inform DAs only (ViGGO inf),\nwith the E2E dataset. The average trigram frequency was calculated on trigrams that appear more than once.\nBLEU METEOR ROUGE CIDEr SER\nAo3 0.519 0.388 0.631 2.531 2.55%\nBo3 0.521 0.391 0.638 2.545 2.48%\nTable 4: Baseline system performance on the ViGGO\ntest set. Despite individual models (Bo3 – best of 3 ex-\nperiments) often having better overall scores, we con-\nsider the Ao3 (average 